In [56]:
import pandas as pd

Order_Products_Prior_DF = pd.read_csv('priororder.csv')
ordersDF = pd.read_csv('orders.csv')
productsDF = pd.read_csv('products.csv')

In [57]:

# orders in prior merged with product names
Order_Product_Name_Prior = pd.merge(Order_Products_Prior_DF, 
                                    productsDF, how='left', on='product_id')
# Prior orders with user_id, product_id, product_name
Prior_User_Order_Product = pd.merge(Order_Product_Name_Prior, 
                                    ordersDF, how='left', on='order_id')

Prior_User_Order_Product['product_name'][0]

'furniture_mattress_and_upholstery'

In [58]:
products = Prior_User_Order_Product['product_name']
product_no_space = []
for product in products:
    product = product.replace(" ", "_")
    product_no_space.append(product)

# drop original column, replace it with one with no space
Prior_User_Order_Product.drop(['product_name'], axis=1)
Prior_User_Order_Product['product_name'] = product_no_space

In [59]:
# add product name to each user
name_list = []
for p_name in Prior_User_Order_Product.groupby('order_id')['product_name']:
    name_list.append(' '.join(p_name[1]))
    
order_id = Prior_User_Order_Product.groupby('order_id')['product_name'].agg('count').index
order_products = pd.DataFrame({'order_id':order_id, 'products':name_list})

In [60]:
order_products.head()


,order_id,products
0,2,furniture_mattress_and_upholstery furniture_de...
1,3,furniture_decor home_comfort small_appliances ...
2,4,furniture_decor home_comfort home_comfort home...
3,5,fashion_shoes fashion_beach fashion_bags_acess...
4,6,electronics computers_acessories computers


In [61]:
order_products.tail()


,order_id,products
281,296,fashion_shoes fashion_bags_acessories fashion_...
282,297,fashion_sport fashion_bags_acessories fashion_...
283,298,construction_tools home_construction construct...
284,299,garden_tools construction_tools_lights constru...
285,300,pet_shop signalling_and_security security_and_...


In [62]:
dataFrameList = []
index = 0
for row in order_products['products']:
    productsName = row.split(' ')
    tup = (index, productsName)
    dataFrameList.append(tup)
    index += 1

In [63]:
# randomly split data into train (70%) and test (30%)
import random
import numpy
random.shuffle(dataFrameList)

train_data = dataFrameList[:400]
test_data = dataFrameList[400:]

In [64]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec

spark = SparkSession.builder.appName("Bigram").getOrCreate()

# spark dataframe： read in batch of 10000 due to large computation
N = len(train_data)//10000
mod = len(train_data) % 10000
trainDF = spark.createDataFrame(dataFrameList[0:10000], ['id',"product_name"])

for i in range(1,N):
    trainDF_sub = spark.createDataFrame(train_data[10000*i:10000*(i+1)], ['id',"product_name"])
    traintDF = trainDF.union(trainDF_sub)
    
trainDF_sub = spark.createDataFrame(train_data[10000*N:len(train_data)], ['id',"product_name"])
trainDF = trainDF.union(trainDF_sub)

In [65]:
from pyspark.ml.feature import NGram
ngram = NGram(n=2, inputCol="product_name", outputCol="bigrams")
ngramDataFrame = ngram.transform(trainDF)
ngramDataFrame.head()

Row(id=225, product_name=['arts_and_craftmanship', 'cool_stuff', 'art', 'cool_stuff', 'photo_frame', 'art', 'baby', 'toys', 'diaper_and_hygiene', 'diaper_and_hygiene', 'diaper_and_hygiene', 'baby', 'baby', 'baby', 'diaper_and_hygiene', 'diaper_and_hygiene', 'baby', 'baby'], bigrams=['arts_and_craftmanship cool_stuff', 'cool_stuff art', 'art cool_stuff', 'cool_stuff photo_frame', 'photo_frame art', 'art baby', 'baby toys', 'toys diaper_and_hygiene', 'diaper_and_hygiene diaper_and_hygiene', 'diaper_and_hygiene diaper_and_hygiene', 'diaper_and_hygiene baby', 'baby baby', 'baby baby', 'baby diaper_and_hygiene', 'diaper_and_hygiene diaper_and_hygiene', 'diaper_and_hygiene baby', 'baby baby'])

In [66]:
bigrams = ngramDataFrame.toPandas()['bigrams']
table = {}
total = len(bigrams)
completion = 0
for bigram in bigrams:
    for combination in bigram:
        components = combination.split(' ')
        key = components[0]
        valKey = components[1]
        if key in table:
            valueDict = table[key]
            if valKey in valueDict:
                valueDict[valKey] = valueDict[valKey] + 1
            else:
                valueDict[valKey] = 1
        else:
            # create new value for key
            valueDict =  {valKey: 1}
            table[key] = valueDict
    completion += 1

In [67]:
for firstWord in table:
    for secondWord in table[firstWord]:
        if table[firstWord][secondWord] > 20:
            print(firstWord, " + ", secondWord, ": ", table[firstWord][secondWord])

baby  +  toys :  38
baby  +  baby :  40
baby  +  diaper_and_hygiene :  46
toys  +  diaper_and_hygiene :  40
toys  +  baby :  34
toys  +  toys :  38
diaper_and_hygiene  +  diaper_and_hygiene :  54
diaper_and_hygiene  +  baby :  44
diaper_and_hygiene  +  toys :  44
electronics  +  electronics :  22
fashion_shoes  +  fashion_bags_acessories :  22
construction_tools_safety  +  construction_tools_lights :  28
christmas_supplies  +  drinks :  22
party_supplies  +  party_supplies :  22
pet_shop  +  pet_shop :  44
home_construction  +  construction_tools :  22
home_construction  +  home_construction :  28
construction_tools  +  home_construction :  24
construction_tools  +  construction_tools :  22
health_beauty  +  perfumery :  104
health_beauty  +  health_beauty :  94
perfumery  +  perfumery :  90
perfumery  +  health_beauty :  102
construction_tools_lights  +  home_construction :  22
signalling_and_security  +  signalling_and_security :  122
signalling_and_security  +  security_and_services

In [68]:
def getPureData(prodName):
    
    '''sort the bigram frequencies in descending order, 
       then return merely the corresponding product names in the same order'''
    
    if prodName not in table:
        return []
    sortedOringalList = sorted(table[prodName].items(), key=lambda x: x[1], reverse=True)
#     print(sortedOringalList)
    data = {}
    for tp in sortedOringalList:
        product = tp[0]
        number = tp[1]
        if number in data:
            productList = data[number]
            productList.append(product)
        else:
            productList = [product]
        data[number] = productList
#     print(data)
#     print("==> Get pure data name:")
    pureData = data.values()
#     print(pureData)
    return list(pureData)
def pickRecommendProds(pureData, numberOfRecommend):
    
    '''Pick certain number of products from the sorted product names'''
    
    recommendProds = []
    for prods in pureData:
        if len(prods) <= numberOfRecommend:
            recommendProds += prods
            numberOfRecommend -= len(prods)
        else:
            recommendProds += random.sample(prods, numberOfRecommend)
            numberOfRecommend = 0

        if numberOfRecommend == 0:
            break
    
    return recommendProds

# recommend products bought together with 'name'
# name: the product to start with
def getRecommend(name, numberOfRecommend):
    
    '''Recommend certain number of products bought after the given input name'''
    
    # numberOfRecommend = 10
    recommendProducts = []
    productName = name
    index = 0
    
    while (numberOfRecommend):
#         print("->Target: ", productName)
#         print("->numberOfRecommend: ", numberOfRecommend)
#         print("->Index: ", index)
        data = getPureData(productName)
    #     print("Pure data:", data)
        intermediate = pickRecommendProds(data, numberOfRecommend)
        recommendProducts += intermediate
#         print("Recommend: ", recommendProducts)
#         print("Recommend: ", recommendProducts)
        if len(intermediate) == 0 and index == len(recommendProducts):
            break
        numberOfRecommend -= len(intermediate)
        if numberOfRecommend > 0:
#             print("Still left: ", numberOfRecommend)
            productName = recommendProducts[index]
            index += 1

#         print("==================")

    return recommendProducts

In [70]:
print(getRecommend("computers", 5))

['electronics', 'consoles_games', 'tablets', 'computers_acessories', 'computers']
